In [4]:
from bs4 import BeautifulSoup as bs
import requests
import json
import pdfminer
from pdfminer.pdfparser import PDFSyntaxError
import os
import re
ROOT_URL = "https://www.drugs.com/answers/questions/"


def page_soup(url):
    """return a soup object"""
    html = requests.get(url).text
    return bs(html, 'html.parser')


def get_pdf_hrefs(url):
    pdf_href_list = []  # {page_num: [BULAs URIs]}
    soup = page_soup(url)

    pdf_link_list = soup.findAll('a')
    for link in pdf_link_list:
        if re.search('/answers/[^/]*\.html',link.attrs['href']):

            pdf_href_list.append(link.attrs['href'])
    return pdf_href_list

def dump_list_to_json_file(list_, file_name):
    with open(file_name, 'w') as fp:
        json.dump(list_, fp, indent=4)


for a in range(0,9):
    href_list = []
    for b in range(1 + a*819, 819 + a*819):
        query = '?page=' + str(b)
        href_list += get_pdf_hrefs(ROOT_URL + query)
    file_name = 'hreflist' + str(a) + '.json'
    dump_list_to_json_file(href_list, file_name)

In [10]:
def read_list_from_json(filename):
    with open(filename) as infile:
        j = json.load(infile)
    return j

def get_sentences(url):
    base = "https://www.drugs.com"
    soup = page_soup(base + url)
    pat = re.compile('.*\(([0-9]+)\).*')
    resphead = soup.find('div', attrs={'class':'replyListHeader'})
    mat =re.match(pat, resphead.text.strip('\n'))
    postcontents = soup.find('div', attrs={'class':'postContent'})
    if postcontents:
        responses = [postcontents.text.replace('\n','')]
        if mat:
            num = int(mat.group(1))
            listcontents = soup.findAll('div', attrs={'class':'listContent'})
            responses += [listcontents[x].text.replace('\n','').strip('Votes:[+\-0-9] Comment Vote up Report') for x in range(1,num + 1)]
        return responses
    else:
        return[]

In [13]:
for a in range(2,9):
    file_name = 'hreflist' + str(a) + '.json'
    sfile_name = 'sentencelist' + str(a) + '.json'
    
    href_list = read_list_from_json(file_name)
    print(len(href_list))
    sentencelist = []
    for url in href_list:
        try:
            sentencelist += get_sentences(url)
        except BaseException as error:
            print('An exception occurred: {}'.format(error))
            continue
    dump_list_to_json_file(sentencelist, sfile_name)


16155
17153
16298
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
15794
An exception occurred: list index out of range
An exception occurred: list index out of range
15766
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
15784
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: l

In [16]:
from nltk.corpus import stopwords
import string
def removepuncandsplit(sentence):
    p=string.punctuation
    stopWords = set(stopwords.words('english'))

    for x in p:
        sentence = sentence.replace(x,'') 
    word_list = sentence.split(' ')
    filtered_words = [word for word in word_list if word not in stopWords]
    return filtered_words

from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
def sentencetokenize(text):
    punkt_param = PunktParameters()
    punkt_param.abbrev_types = set(['dr', 'vs', 'mr', 'mrs', 'prof', 'inc'])
    sentence_splitter = PunktSentenceTokenizer(punkt_param)
    sentences = sentence_splitter.tokenize(text)
    tokdsentences = []
    for sentence in sentences:
        tokdsentences.append(removepuncandsplit(sentence))
    return tokdsentences
        


In [18]:
sentencelist = []
for a in range(0,9):
    sfile_name = 'sentencelist' + str(a) + '.json'
    
    sentencelist += read_list_from_json(sfile_name)

sentences = []
for i,sentence in enumerate(sentencelist):
    if i%10000 == 0:
        print(i)
    sentences += sentencetokenize(sentence)
    


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000


In [20]:
from gensim.models import Word2Vec
min_count = 5
size = 50
window = 5
 
model = Word2Vec(sentences, min_count=min_count, size=size, window=window)

D:\apps\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [19]:
len(sentences)

1646221

In [60]:
model.most_similar('carbohydrate')

D:\apps\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('fats', 0.8851694464683533),
 ('carbohydrates', 0.8768407106399536),
 ('fibre', 0.8678574562072754),
 ('protein', 0.8522319793701172),
 ('carb', 0.8521303534507751),
 ('saturated', 0.8519685864448547),
 ('caloric', 0.8446818590164185),
 ('nutritious', 0.8128793239593506),
 ('salads', 0.8053873181343079),
 ('refined', 0.8044947385787964)]

In [42]:
import pickle
output = open('faqmodel.pkl', 'wb')
pickle.dump(model, output)

In [104]:
len(list(model.wv.vocab.keys()))

49494

In [56]:
from gensim.models.keyedvectors import KeyedVectors


In [58]:
KeyedVectors.save('faqmodel.pkl',binary=True,encoding='pickle')

LookupError: unknown encoding: pickle

In [80]:
model.wv.save_word2vec_format('faqmodel.bin', binary=True)